In [1]:
pip install stanza


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer 
#import stanfordnlp
import stanza
import string

In [3]:
stanza.download('en') 

2022-10-18 18:13:24 INFO: Downloading default packages for language: en (English) ...
2022-10-18 18:13:27 INFO: File exists: C:\Users\msuryana\stanza_resources\en\default.zip
2022-10-18 18:13:36 INFO: Finished downloading models and saved to C:\Users\msuryana\stanza_resources.


In [4]:
#stanfordnlp.download('en')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\msuryana\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\msuryana\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\msuryana\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [5]:
#nlp = stanfordnlp.Pipeline()
nlp = stanza.Pipeline('en')

2022-10-18 18:13:37 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2022-10-18 18:13:40 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2022-10-18 18:13:40 INFO: Use device: cpu
2022-10-18 18:13:40 INFO: Loading: tokenize
2022-10-18 18:13:40 INFO: Loading: pos
2022-10-18 18:13:41 INFO: Loading: lemma
2022-10-18 18:13:41 INFO: Loading: depparse
2022-10-18 18:13:41 INFO: Loading: sentiment
2022-10-18 18:13:42 INFO: Loading: constituency
2022-10-18 18:13:44 INFO: Loading: ner
2022-10-18 18:13:45 INFO: Done loading processors!


In [6]:
# Make sure you have downloaded the StanfordNLP English model and other essential tools using,
# stanfordnlp.download('en')
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

def aspect_sentiment_analysis(txt, stop_words, nlp):
    
    txt = txt.lower() # LowerCasing the given Text
    sentList = nltk.sent_tokenize(txt) # Splitting the text into sentences

    fcluster = []
    totalfeatureList = []
    finalcluster = []
    dic = {}

    for line in sentList:
        newtaggedList = []
        txt_list = nltk.word_tokenize(line) # Splitting up into words
        taggedList = nltk.pos_tag(txt_list) # Doing Part-of-Speech Tagging to each word

        newwordList = []
        flag = 0
        for i in range(0,len(taggedList)-1):
            if(taggedList[i][1]=="NN" and taggedList[i+1][1]=="NN"): # If two consecutive words are Nouns then they are joined together
                newwordList.append(taggedList[i][0]+taggedList[i+1][0])
                flag=1
            else:
                if(flag==1):
                    flag=0
                    continue
                newwordList.append(taggedList[i][0])
                if(i==len(taggedList)-2):
                    newwordList.append(taggedList[i+1][0])

        finaltxt = ' '.join(word for word in newwordList) 
        new_txt_list = nltk.word_tokenize(finaltxt)
        wordsList = [w for w in new_txt_list if not w in stop_words]
        wordsList = [w for w in wordsList if not w in string.punctuation]
        taggedList = nltk.pos_tag(wordsList)
        #print(finaltxt)

        #doc = nlp(txt) # Object of Stanford NLP Pipeline
        doc = nlp(finaltxt) # Object of Stanford NLP Pipeline
        #print(doc)
        
        # Getting the dependency relations betwwen the words
        dep_node = []
        for dep_edge in doc.sentences[0].dependencies:
            #print(dep_edge[0].text)
            #print(dep_edge[1])
            #print("------------------------------")
            if(not dep_edge[2].text in string.punctuation):
                dep_node.append([dep_edge[2].text, dep_edge[0].id, dep_edge[1]])

        #print(len(newwordList))
        #print(len(dep_node))
        # Coverting it into appropriate format
        for i in range(0, len(dep_node)):
            if (int(dep_node[i][1]) != 0):
                #print(int(dep_node[i][1])
                try:
                    dep_node[i][1] = newwordList[(int(dep_node[i][1]) - 1)]
                except:
                    dep_node[i][1] = ""

        featureList = []
        categories = []
        #print(taggedList)
        for i in taggedList:
            if(i[1]=='JJ' or i[1]=='NN' or i[1]=='JJR' or i[1]=='NNS' or i[1]=='RB'):
                featureList.append(list(i)) # For features for each sentence
                totalfeatureList.append(list(i)) # Stores the features of all the sentences in the text
                categories.append(i[0])

        for i in featureList:
            filist = []
            for j in dep_node:
                #print(j)
                if((j[0]==i[0] or j[1]==i[0]) and (j[2] in ["nsubj", "acl:relcl", "obj", "dobj", "agent", "advmod", "amod", "neg", "prep_of", "acomp", "xcomp", "compound"])):
                    if(j[0]==i[0]):
                        filist.append(j[1])
                    else:
                        filist.append(j[0])
            fcluster.append([i[0], filist])
            
    #print(fcluster)
    for i in totalfeatureList:
        dic[i[0]] = i[1]
    
    for i in fcluster:
        if(dic[i[0]]=="NN"):
            finalcluster.append(i)
        
    return(finalcluster)

In [7]:
stop_words = set(stopwords.words('english'))

In [8]:
# Import reviews data
yelp_reviews = pd.read_feather('../../FilteredData/review_philly.feather')
yelp_reviews_filtered = yelp_reviews[['review_id', 'user_id', 'business_id', 'text', 'stars']]
yelp_reviews_filtered['absa_pred']=yelp_reviews_filtered['text']

<ipython-input-8-73aa92dc8c22>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yelp_reviews_filtered['absa_pred']=yelp_reviews_filtered['text']


In [9]:
for i in range(yelp_reviews_filtered.shape[0]):
    txt = yelp_reviews_filtered['text'][i]
    try:
        yelp_reviews_filtered['absa_pred'][i] = aspect_sentiment_analysis(txt, stop_words, nlp)
    except:
        yelp_reviews_filtered['absa_pred'][i] = "FAILED"

<ipython-input-9-33fdfd123100>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yelp_reviews_filtered['absa_pred'][i] = aspect_sentiment_analysis(txt, stop_words, nlp)
C:\Users\msuryana\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [10]:
yelp_reviews_filtered.to_csv('predicted_absa.csv')